In [8]:
%config Completer.use_jedi = False

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
#import geoplot as gplt
import geopandas as gpd
import os
import copy
from copy import deepcopy
from shapely.geometry import Point, Polygon
from jmetal.util.constraint_handling import overall_constraint_violation_degree
from scipy.spatial.distance import squareform 
from scipy.spatial.distance import pdist
from jmetal.util.observer import ProgressBarObserver
from jmetal.util.constraint_handling import *
from Problem.JMetalPy_modified import *
from Problem.ConstraintEvaluation import *
from Problem.PerformanceObserver import *
from Problem.InitialSolution import *
from Problem.FitnessEvaluation import *
from Problem.PlotFunctions import *
from Problem.RoutingContext import *
from Problem.VehicleFunctions import *
from Problem.VRP import *
from Problem.utils import *
from Problem.Congestion_VRP import *
from Problem.GIFGenerator import *
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working#40544668
api = "Tfmt6Sxyn-nfinSHUyUo3xmVDWdtwAUyh6C8Y3HTZhU"

dataset = "UK25_2"
meta_data, distance_data, station_data = parse_UK_Data("../datasets/{}/{}.txt".format(dataset[:-2],dataset))

station_coordinates = pd.read_csv('../datasets/{}/{}_coordinates.csv'.format(dataset[:-2],dataset), encoding='utf-8')
points_coordinate = station_coordinates[["lat", "lng"]].to_numpy()
dbf = gpd.GeoDataFrame.from_file(u"../datasets/united_kingdom_european_regions/european_region_region.dbf")
dbf = dbf.to_crs("EPSG:4326")
meta_data["F-C Empty (l/100km)"] = 12.5
meta_data["F-C Full (l/100km)"] = 15
meta_data["Vehicle Weight"] = 3000

distance_matrix_here,time_matrix_here = load_saved_here_data("../datasets/{}/".format(dataset[:-2]),dataset)

distance_matrix_here = distance_matrix_here.to_numpy()
time_matrix_here = time_matrix_here.to_numpy()

routingContext = RoutingContext(distance_matrix=distance_matrix_here,time_matrix=time_matrix_here,station_coordinates=station_coordinates,station_data=station_data,meta_data=meta_data)
startNodes = {"UK20_1":[0,6,10,15],"UK25":[0,6,10,15],"UK50":[0,6,10,15,21,35,40],"UK75":[0,6,10,15,21,35,40,60,31,68]}
endNodes = {"UK20_1":[4,20,2,5],"UK25":[4,20,2,5,22],"UK50":[4,20,2,5,32,45,1],"UK75":[4,20,2,5,32,45,1]}


ibea_25_2_path = [['V0', 14, 9, 13, 2], ['V1', 7, 23, 22], ['V3', 11, 18, 3, 2], ['V4', 8, 19, 20], ['V2', 21, 1, 24, 2], ['V5', 25, 16, 12, 17, 4]]
ibea_25_2_multiobjective_path = [['V0', 14, 9, 24, 13, 7, 23, 22], ['V4', 1, 21, 8, 19, 20], ['V5', 25, 11, 18, 3, 16, 12, 17, 4]]
ibea_50_1_path = [['V0', 42, 33, 30, 19, 34, 16, 2], ['V3', 24, 50, 26, 27, 2], ['V2', 48, 31, 2], ['V4', 11, 18, 39, 20], ['V6', 47, 8, 46, 2], ['V8', 44, 7, 37, 2], ['V9', 36, 17, 25, 13, 14, 29, 38, 2], ['V10', 41, 45], ['V12', 49, 3, 12, 23, 9, 28, 20], ['V13', 22, 43, 20]]

In [2]:
startNodeIndeces = startNodes[dataset[:-2]]
endNodeIndeces = endNodes[dataset[:-2]]
locationNodeIndeces = list((set(range(len(points_coordinate))) - set(startNodeIndeces)) - set(endNodeIndeces))

#vehicleData = {"emptyWeights":[3000],"maxLoads":[3650],"fuelEmpty":[12],"fuelFull":[15]}
vehicleData = {"emptyWeights":[3000,5500],"maxLoads":[3650,8500],"fuelEmpty":[12,22.5],"fuelFull":[15,27.5]}


vehicles = initializeVehicles(startIndices=startNodeIndeces,vehicleData=vehicleData,heterogeneousFleet=True)

customer_demands = station_data.T.loc["Demand(kg)"].astype(int).to_numpy()
customer_demands[startNodeIndeces+endNodeIndeces] = 0
station_data["Demand(kg)"] = customer_demands
#station_data.iloc[startNodeIndeces+endNodeIndeces,"Demand(kg)"] = 0

translate_dict = {}
for i,x in enumerate(endNodeIndeces):
    translate_dict[x] = 100+i
initial_solution = cheapest_insertion_dict(locationNodeIndeces,vehicles,endNodeIndeces,routingContext)


min_allowed_drivetime_hours = 0

cost_function_dict = {"distance_matrix":distance_matrix_here,"time_matrix":time_matrix_here,
                              "demands":customer_demands,"vehicle_weight":3000,"start_positions":startNodeIndeces,"meta_data":meta_data}

#old_init_sol = create_initial_solution(locationNodeIndeces,vehicles,endNodeIndeces,fuel_consumption_wong,cost_function_dict)

#initial_solution_no_starts =initial_solution_kmeans(locationNodeIndeces,endNodeIndeces,points_coordinate,translate_dict)# 
data_model_input_multi_objective = {'routing_context':routingContext,'vehicles':vehicles,'end_points':endNodeIndeces,'pickup_points':locationNodeIndeces,
                                    'initial_solution':initial_solution,
                                    'objective_amount':2,'objective_labels':['Total Fuel','Total Drive Time'],
                                    'constraint_amount':5,'number_of_cities':routingContext.distance_matrix.shape[0],
                                   'min_drivetime':min_allowed_drivetime_hours}

data_model_input_single_objective = {'routing_context':routingContext,'vehicles':vehicles,'end_points':endNodeIndeces,'pickup_points':locationNodeIndeces,
                                    'initial_solution':initial_solution,
                                    'objective_amount':1,'objective_labels':['Total Fuel'],
                                    'constraint_amount':5,'number_of_cities':routingContext.distance_matrix.shape[0],
                                    'min_drivetime':min_allowed_drivetime_hours}

problem = VRP(data_model_input_multi_objective)

plot_data_storage_base = {'fitness':[], 'route_distance':[],'fuel_consumption':[],'vehicle_loads':[],'computation_times':[],'vehicle_route_time':[],'violation':[],'paths':[],'distance_to_origin':[]}


In [3]:
def plot_routes(vehicle_solutions, points_coordinate, dbf,station_ids = False, here_api = False, api_key=""):
    fig, ax = plt.subplots(figsize=(30,30))# add .shp mapfile to axes
    dbf.plot(ax=ax, alpha=0.4,color="grey")
    
    for vehicle_id in range(len(vehicle_solutions)):
        vehicle_stops_coordinates=points_coordinate[vehicle_solutions[vehicle_id], :]
        if here_api:
            vehicle_route_encoded = generate_routes(vehicle_stops_coordinates, api_key=api_key)
            vehicle_route_decoded = decode_routes(vehicle_route_encoded)
            total_vehicle_route = []
            for route in vehicle_route_decoded:
                for cords in route:
                    total_vehicle_route.append(cords)
            total_vehicle_route = np.array(total_vehicle_route)
            
            ax.plot(total_vehicle_route[:, 1], total_vehicle_route[:, 0])
        else:
            ax.plot(vehicle_stops_coordinates[:, 1], vehicle_stops_coordinates[:, 0])
        
        if station_ids:
            for i, (x, y) in zip(vehicle_solutions[vehicle_id],
                                 zip(vehicle_stops_coordinates[:, 1],
                                     vehicle_stops_coordinates[:, 0])):

                ax.text(x, y, str(i), color="red", fontsize=12)
                ax.plot()
        else:
            for i, (x, y) in enumerate(zip(vehicle_stops_coordinates[:, 1],
                                     vehicle_stops_coordinates[:, 0])):
                ax.text(x, y, str(i), color="red", fontsize=12)
    return fig,ax

In [4]:
def preprocess_vehicle_paths(numerical_paths,coordinates,api_key):
    prerocessed_paths = []
    for vehicle_id in range(len(numerical_paths)):

            vehicle_stops_coordinates=points_coordinate[numerical_paths[vehicle_id], :]
            vehicle_route_encoded = generate_routes(vehicle_stops_coordinates, api_key=api)
            vehicle_route_decoded = decode_routes(vehicle_route_encoded)
            total_vehicle_route = []
            
            for route in vehicle_route_decoded:
                route_cords = []
                for cords in route:
                    route_cords.append(cords)
                total_vehicle_route.append(route_cords)
            prerocessed_paths.append(np.array(total_vehicle_route))
            
    return prerocessed_paths
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import scipy as sp
from copy import deepcopy
def get_vehicle_color_map2(vehicles):
    
    color_list = deepcopy(mcolors.TABLEAU_COLORS)
    del color_list["tab:red"]
  
    colors = list(color_list.values())
    colors.append("k")
    color_map = {}

    for vehicle_key,color in zip(vehicles,colors):
        color_map[vehicle_key] = color
    return color_map
color_map = get_vehicle_color_map2(vehicles)
#preprocessed_paths_normal= preprocess_vehicle_paths(nsga3_numerical_path,points_coordinate,api)

In [5]:
def create_congestion(vehicle_paths,vehicles):
    
    numerical_paths = get_numerical_path(vehicle_paths,vehicles)
    paths_sorted_length = sorted(numerical_paths,key=lambda x: len(x),reverse=True)
    
    longest_path = random.sample(numerical_paths,k=1)[0]
    print(longest_path)
    
    #longest_path = paths_sorted_length[0]
    longest_path_length = len(longest_path)
    congestion_point = random.randint(1,longest_path_length - 2)
    congestion_edge = (longest_path[congestion_point-1],longest_path[congestion_point])
    longest_path_shifted = np.roll(longest_path,-1)
    drivetimes = time_matrix_here[longest_path[:-1],longest_path_shifted[:-1]]
    congestion_time = np.cumsum(drivetimes)[congestion_point]
    congestion_time
    locked_paths = {}
    for path in vehicle_paths:

        locked_paths[path[0]] = [path[0]]
        vehicle_index = [vehicles[path[0]]["startPos"]]
        numerical_route = vehicle_index +  path[1:]
        numerical_route_shifted = np.roll(numerical_route,-1)

        drive_times = time_matrix_here[numerical_route,numerical_route_shifted]
        drive_times[-1] = 0
        drive_times_cumul = np.cumsum(drive_times)
        test = len(list(filter(lambda x: x < congestion_time, drive_times_cumul)))

        edge_drive_times = list(zip(numerical_route,drive_times_cumul))
        locked_paths[path[0]].extend(path[1:test])

    unused_vehicles = list(filter(lambda x: not x in locked_paths.keys(),vehicles.keys()))
    for unused_vehicle in unused_vehicles:
        locked_paths[unused_vehicle] = [unused_vehicle]

    for index,vehicle in enumerate(locked_paths):

        if locked_paths[vehicle][-1] in endNodeIndeces:
            locked_paths[vehicle].pop()

    current_sol = [x if not x in endNodeIndeces else -1 - index for index, x in enumerate(list(chain(*vehicle_paths)))]
    locked_pickups = list(filter(lambda x: type(x) != str and not x in endNodeIndeces,chain(*locked_paths.values())))

    return congestion_edge,locked_pickups,current_sol,locked_paths
 

In [16]:
import folium
from folium import Choropleth, Circle, Marker,PolyLine,CircleMarker
from folium.plugins import HeatMap, MarkerCluster
def plot_routes_with_congestion_folium(vehicle_solutions_path, points_coordinate, dbf,colors,congestion_edge=None,locked_paths=None,station_ids = False,prerocessed_paths=None, here_api = False, api_key=""):
    
    vehicle_solutions = get_numerical_path(vehicle_solutions_path,vehicles)
    
    folium_map = folium.Map(np.mean(points_coordinate,axis=0),zoom_control=False  ,zoom_start=10,min_zoom=10)#,tiles = "cartodbdark_matter")
    icons = {0:"trash",1:"industry",2:"truck",3:'car'}
    type_colors=["black","red","green"]
    
    
    congestion_plot_coordinates = np.array([points_coordinate[congestion_edge[0]],points_coordinate[congestion_edge[1]]]) if congestion_edge else None
    print(congestion_plot_coordinates)
        
    for vehicle_id in range(len(vehicle_solutions)):
        plot_color = colors[list(colors.keys())[vehicle_id]]
        
        vehicle_stops_coordinates=points_coordinate[vehicle_solutions[vehicle_id], :]
        vehicle_route = vehicle_stops_coordinates
        
        if here_api:
            if type(prerocessed_paths) != None:
                vehicle_route = prerocessed_paths[vehicle_id]
            else:
                vehicle_route_encoded = generate_routes(vehicle_stops_coordinates, api_key=api_key)
                vehicle_route_decoded = decode_routes(vehicle_route_encoded)
                total_vehicle_route = []
                for route in vehicle_route_decoded:
                    for cords in route:
                        total_vehicle_route.append(cords)
                vehicle_route = np.array(total_vehicle_route)
        
        PolyLine(vehicle_route,weight=5,color=plot_color).add_to(folium_map)
        
        
        
    planted_start_stop_ids=[]
    for vehicle_id in range(len(vehicle_solutions)):
        
        locked_vehicle_path = get_numerical_path([locked_paths[vehicle_solutions_path[vehicle_id][0]]],vehicles) if congestion_edge else [-1]
        
        
        plot_color =colors[list(colors.keys())[vehicle_id]]
        
        vehicle_stops_coordinates=points_coordinate[vehicle_solutions[vehicle_id], :]
        vehicle_route = vehicle_stops_coordinates
        vehicle_string = vehicle_solutions_path[vehicle_id][0]
        
    
    
        for stop_index, vehicle_stop in enumerate(vehicle_stops_coordinates):
            stop_coordinates = (vehicle_stop[0],vehicle_stop[1])
            node_type = 0
            
            if stop_index == 0:
                if int(vehicle_string[1:]) % 2 ==0:
                    node_type = 2
                else:
                    node_type = 3
                
            elif stop_index == len(vehicle_stops_coordinates)-1:
                node_type = 1
                
            pos_icon = icons[node_type]
            
            if node_type == 0:
                CircleMarker(stop_coordinates,color="#4b4d4d",fillColor="#4b4d4d",fill_opacity=.9,fill=True,radius=5).add_to(folium_map)
            else:
                
                beautiful_icon = folium.plugins.BeautifyIcon(color="white",iconSize=(35,35),inner_icon_style='font-size:20px',innerIconAnchor=(0,5),text_color=plot_color,icon=pos_icon,prefix="fa",border_width=2)
                offset=np.array([0,0])
                current_pos  = vehicle_solutions[vehicle_id][stop_index]
                
                occurence_offsets = {1:np.array([0.008,0.01]),2:np.array([0.008,-0.01]),3:np.array([-0.012,0.01]),4:np.array([-0.012,-0.02]),5:np.array([0,0.025])}
                
                
                if current_pos in planted_start_stop_ids and node_type in [2,3]:
                    offset=+np.array([0.008,0.01])
                    
                elif current_pos in planted_start_stop_ids and node_type ==1 :
                    offset= occurence_offsets[planted_start_stop_ids.count(current_pos)]
                    print(planted_start_stop_ids.count(current_pos), plot_color)
#                     amount_of_ends = planted_start_stop_ids.count(current_pos)
#                     test_icon = folium.plugins.BeautifyIcon(color="white",iconSize=(35,35),inner_icon_style='font-size:24px',innerIconAnchor=(0,-2),text_color=plot_color,number=amount_of_ends,prefix="fa",border_width=2)
#                     Marker(tuple(np.array(stop_coordinates)+np.array([0.0,0.04])),icon=test_icon).add_to(folium_map)
                    
                planted_start_stop_ids.append(current_pos)
                Marker(tuple(np.array(stop_coordinates)+ offset)  ,icon=beautiful_icon).add_to(folium_map)
                    
                    
                
        #planted_start_stop_ids.append(vehicle_solutions[vehicle_id][0])
        #planted_start_stop_ids.append(vehicle_solutions[vehicle_id][-1])
        
        
        
        
    if congestion_edge != None:
        congestion_route =congestion_plot_coordinates
        if here_api:
        
        
            congestion_route_encoded = generate_routes(congestion_plot_coordinates, api_key=api_key)
            congestion_route_decoded = decode_routes(congestion_route_encoded)
            total_congestion_route = []
            for route in congestion_route_decoded:
                    for cords in route:
                        total_congestion_route.append(cords)
            congestion_route = np.array(total_congestion_route)

        PolyLine(congestion_route,color="red",weight=6,dash_array='10').add_to(folium_map)
    
    #max_point =  [max(points_coordinate[:,0]), max(points_coordinate[:,1])]
    #min_point =  [min(points_coordinate[:,0]), min(points_coordinate[:,1])-0.6]
    #folium_map.fit_bounds([min_point,max_point])
    
#     if congestion_edge:
#         folium_map.position =(5,5)# np.mean([points_coordinate[congestion_edge[0]], points_coordinate[congestion_edge[1]]],axis=0) + np.array([1,-5])
    
    
    
    #folium_map.position = tuple(np.mean([max_point,min_point],axis=0))+(1,5)
    return folium_map

In [6]:
# import folium
# from folium import Choropleth, Circle, Marker,PolyLine,CircleMarker
# from folium.plugins import HeatMap, MarkerCluster
# def plot_routes_with_congestion_folium(vehicle_solutions_path, points_coordinate, dbf,colors,congestion_edge=None,locked_paths=None,station_ids = False,prerocessed_paths=None, here_api = False, api_key=""):
    
#     vehicle_solutions = get_numerical_path(vehicle_solutions_path,vehicles)
    
#     folium_map = folium.Map(np.mean([points_coordinate[congestion_edge[0]], points_coordinate[congestion_edge[1]]],axis=0),zoom_control=False  ,zoom_start=11,min_zoom=11)#,tiles = "cartodbdark_matter")
#     icons = {0:"trash",1:"industry",2:"truck",3:'car'}
#     type_colors=["black","red","green"]
    
    
#     congestion_plot_coordinates = np.array([points_coordinate[congestion_edge[0]],points_coordinate[congestion_edge[1]]]) if congestion_edge else None
#     print(congestion_plot_coordinates)
        
#     for vehicle_id in range(len(vehicle_solutions)):
#         plot_color = colors[list(colors.keys())[vehicle_id]]
#         locked_vehicle_path = get_numerical_path([locked_paths[vehicle_solutions_path[vehicle_id][0]]],vehicles)[0] if congestion_edge else [-1]
#         print(vehicle_solutions[vehicle_id])
#         print(locked_vehicle_path)
#         vehicle_stops_coordinates=points_coordinate[vehicle_solutions[vehicle_id], :]
#         vehicle_route = vehicle_stops_coordinates
        
#         if here_api:
#             if type(prerocessed_paths) != None:
#                 vehicle_route = prerocessed_paths[vehicle_id]
#             else:
#                 vehicle_route_encoded = generate_routes(vehicle_stops_coordinates, api_key=api_key)
#                 vehicle_route_decoded = decode_routes(vehicle_route_encoded)
#                 total_vehicle_route = []
#                 for route in vehicle_route_decoded:
#                     for cords in route:
#                         total_vehicle_route.append(cords)
#                 vehicle_route = np.array(total_vehicle_route)
        
#         print(len(locked_vehicle_path))
        
#         print(len(vehicle_route))
      
    
#         locked_len = len(locked_vehicle_path) -1 if len(locked_vehicle_path) >1 else 0
#         if locked_len > 0:
#             PolyLine(vehicle_route[:locked_len],weight=5,color=plot_color).add_to(folium_map)
#         PolyLine(vehicle_route[locked_len:],weight=5,color=plot_color,dash_array='8').add_to(folium_map)
        
#     planted_start_stop_ids=[]
#     for vehicle_id in range(len(vehicle_solutions)):
        
        
        
#         plot_color =colors[list(colors.keys())[vehicle_id]]
        
#         vehicle_stops_coordinates=points_coordinate[vehicle_solutions[vehicle_id], :]
#         vehicle_route = vehicle_stops_coordinates
#         vehicle_string = vehicle_solutions_path[vehicle_id][0]
        
    
    
#         for stop_index, vehicle_stop in enumerate(vehicle_stops_coordinates):
#             stop_coordinates = (vehicle_stop[0],vehicle_stop[1])
#             node_type = 0
#             current_pos  = vehicle_solutions[vehicle_id][stop_index]
#             if stop_index == 0:
#                 if int(vehicle_string[1:]) % 2 ==0:
#                     node_type = 2
#                 else:
#                     node_type = 3
                
#             elif stop_index == len(vehicle_stops_coordinates)-1:
#                 node_type = 1
                
#             pos_icon = icons[node_type]
            
#             if node_type == 0:
                
#                 CircleMarker(stop_coordinates,color="#4b4d4d",fillColor="#4b4d4d",fill_opacity=.9,fill=True,radius=5).add_to(folium_map)
                    
#             else:
                
#                 beautiful_icon = folium.plugins.BeautifyIcon(color="white",iconSize=(35,35),inner_icon_style='font-size:20px',innerIconAnchor=(0,5),text_color=plot_color,icon=pos_icon,prefix="fa",border_width=2)
#                 offset=np.array([0,0])
#                 current_pos  = vehicle_solutions[vehicle_id][stop_index]
                
#                 occurence_offsets = {1:np.array([0.008,0.01]),2:np.array([0.008,-0.01]),3:np.array([-0.012,0.01]),4:np.array([-0.012,-0.02]),5:np.array([0,0.025])}
                
                
#                 if current_pos in planted_start_stop_ids and node_type in [2,3]:
#                     offset=+np.array([0.008,0.01])
                    
#                 elif current_pos in planted_start_stop_ids and node_type ==1 :
#                     offset= occurence_offsets[planted_start_stop_ids.count(current_pos)]
#                     print(planted_start_stop_ids.count(current_pos), plot_color)
# #                     amount_of_ends = planted_start_stop_ids.count(current_pos)
# #                     test_icon = folium.plugins.BeautifyIcon(color="white",iconSize=(35,35),inner_icon_style='font-size:24px',innerIconAnchor=(0,-2),text_color=plot_color,number=amount_of_ends,prefix="fa",border_width=2)
# #                     Marker(tuple(np.array(stop_coordinates)+np.array([0.0,0.04])),icon=test_icon).add_to(folium_map)
                    
#                 planted_start_stop_ids.append(current_pos)
#                 Marker(tuple(np.array(stop_coordinates)+ offset)  ,icon=beautiful_icon).add_to(folium_map)
                    
                    
                
#         #planted_start_stop_ids.append(vehicle_solutions[vehicle_id][0])
#         #planted_start_stop_ids.append(vehicle_solutions[vehicle_id][-1])
        
        
        
        
#     if congestion_edge != None:
#         congestion_route =congestion_plot_coordinates
#         if here_api:
        
        
#             congestion_route_encoded = generate_routes(congestion_plot_coordinates, api_key=api_key)
#             congestion_route_decoded = decode_routes(congestion_route_encoded)
#             total_congestion_route = []
#             for route in congestion_route_decoded:
#                     for cords in route:
#                         total_congestion_route.append(cords)
#             congestion_route = np.array(total_congestion_route)

#         PolyLine(congestion_route,color="red",weight=6,dash_array='10').add_to(folium_map)
    
#     #max_point =  [max(points_coordinate[:,0]), max(points_coordinate[:,1])]
#     #min_point =  [min(points_coordinate[:,0]), min(points_coordinate[:,1])-0.6]
#     #folium_map.fit_bounds([min_point,max_point])
    
# #     if congestion_edge:
# #         folium_map.position =(5,5)# np.mean([points_coordinate[congestion_edge[0]], points_coordinate[congestion_edge[1]]],axis=0) + np.array([1,-5])
    
    
    
#     #folium_map.position = tuple(np.mean([max_point,min_point],axis=0))+(1,5)
#     return folium_map

In [14]:
# preprocces_25_2 = preprocess_vehicle_paths(get_numerical_path(ibea_25_2_path,vehicles=vehicles),vehicles,api)
# preprocces_25_2_multi = preprocess_vehicle_paths(get_numerical_path(ibea_25_2_multiobjective_path,vehicles=vehicles),vehicles,api)

In [339]:
#preprocces_50_1 = preprocess_vehicle_paths(get_numerical_path(ibea_50_1_path,vehicles=vehicles),vehicles,api)

In [10]:
color_map_2 =get_vehicle_color_map2(vehicles)

In [11]:
congestion_edge, locked_pickups,current_sol,locked_paths = create_congestion(ibea_25_2_path,vehicles)
congestion_edge

[0, 7, 23, 22]


(7, 23)

In [19]:
#plot_path = get_numerical_path(ibea_50_1_path,vehicles=vehicles)

test = plot_routes_with_congestion_folium(ibea_25_2_multiobjective_path,points_coordinate,dbf,colors=color_map_2,station_ids = True,prerocessed_paths=preprocces_25_2_multi,here_api=True, api_key=api)
test

None


In [17]:
#plot_path = get_numerical_path(ibea_50_1_path,vehicles=vehicles)

test = plot_routes_with_congestion_folium(ibea_25_2_path,points_coordinate,dbf,colors=color_map_2,station_ids = True,prerocessed_paths=preprocces_25_2,here_api=True, api_key=api)
test

None
1 #2ca02c
2 #8c564b


In [617]:
import selenium.webdriver
#test.fit_bounds([[50.7,-1.7],[52,.7]])
#test.fit_bounds([[55,-4],[56.4,-2]])


# max_point =  [max(points_coordinate[:,0])-.1, max(points_coordinate[:,1])]
# min_point =  [min(points_coordinate[:,0])+.1, min(points_coordinate[:,1])-0.6]
# test.fit_bounds([min_point,max_point])
test.save("map.html")
driver = selenium.webdriver.Chrome()
driver.set_window_size(1500,1500)  # choose a resolution
driver.get('file:///home/andreash/Documents/School/Master_Thesis/Explorer/src/map.html')
time.sleep(1)
driver.save_screenshot('screenshot25_test_congestion_not_solved.png')

True

In [519]:
from jmetal.algorithm.multiobjective.ibea import IBEA
from jmetal.operator.crossover import PMXCrossover,CXCrossover
from jmetal.operator.mutation import PermutationSwapMutation, ScrambleMutation
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.util.evaluator import MultiprocessEvaluator
from jmetal.operator import SBXCrossover, RandomSolutionSelection,BestSolutionSelection,RankingAndCrowdingDistanceSelection
from jmetal.operator.selection import   RouletteWheelSelection
from jmetal.util.solution import get_non_dominated_solutions
from datetime import datetime
optimal_parameters = {"SA":{"mutation":0.9}, "GA":{"mutation":0.9,"crossover":0.25,"population_size":30},
                     "NSGAII":{"mutation":0.9,"crossover":0.50,"population_size":30},
                     "NSGAIII":{"mutation":0.9,"crossover":0.50,"population_size":20},
                     "UNSGAIII":{"mutation":0.9,"crossover":0.25,"population_size":20},
                     "IBEA":{"mutation":0.9,"crossover":0.75,"population_size":20}}
            
parameters_IBEA = optimal_parameters["IBEA"]

plot_data_IBEA = deepcopy(plot_data_storage_base)
for x in range(1):
    problem_IBEA = Congestion_VRP(data_model_input_multi_objective,currentSolution =current_sol,locked_pickup_points=locked_pickups )
    problem_IBEA.set_locked_paths(locked_paths)
    problem_IBEA.set_congestion(congestion_edge,1*60*60)
    
    problem_IBEA.name = "IBEA"
    problem_IBEA.assignClosestEndPoint = True
    max_evaluations = 3000

    algorithm_IBEA = IBEA(problem=problem_IBEA,
                            population_size=parameters_IBEA["population_size"],
                            offspring_population_size=parameters_IBEA["population_size"],
                            kappa =1,
                            mutation=PermutationSwapMutation(parameters_IBEA["mutation"]),
                            crossover=PMXCrossover(parameters_IBEA["crossover"]),
                            termination_criterion=StoppingByEvaluations(max_evaluations),
                            population_evaluator=MultiprocessEvaluator(),
                               )

    performance_observer_IBEA = PerformanceObserver(max_iter=max_evaluations,frequency=1,params=[parameters_IBEA])
    performance_observer_IBEA.maxEpochs= 1
    performance_observer_IBEA.currentEpoch =  x


    algorithm_IBEA.observable.register(observer=performance_observer_IBEA)
    algorithm_IBEA.run()

    solutions_IBEA = algorithm_IBEA.get_result()
    front_IBEA = get_non_dominated_solutions(solutions_IBEA)
    
    best_sol = sorted(front_IBEA,key=lambda solution:np.sum(solution.objectives))[0]
    store_plot_data(plot_data_IBEA,performance_observer_IBEA,best_sol)
fixed_path = best_sol.path

Epoch:1 of 1
Evaluations: 3000/3000
Params: {'mutation': 0.9, 'crossover': 0.75, 'population_size': 20}
Best fitness: 98.0112 --- 308.9                        
Best total fuel consumption:98.01 
Computing time: 4.15s                        
Average computing speed: 723.64it/s
Current Route:[['V0', 14, 9, 13, 2], ['V5', 25, 16, 4], ['V1', 7, 23, 22], ['V3', 11, 3, 18, 2], ['V4', 8, 19, 21, 24, 1, 2], ['V2', 17, 12, 4]]
Flags: []                        
Violation:0
Vehicle amount:6


In [531]:
#plot_path = get_numerical_path(ibea_50_1_path,vehicles=vehicles)

test2 = plot_routes_with_congestion_folium(fixed_path,points_coordinate,dbf,colors=color_map_2,station_ids = True,congestion_edge=congestion_edge, api_key=api)
test2

[[55.7743458 -3.9184134]
 [55.8149158 -3.9376273]]
1 #9467bd
2 #8c564b
1 #e377c2


In [521]:
preprocess_fixed_congestion = preprocess_vehicle_paths(get_numerical_path(fixed_path,vehicles=vehicles),vehicles,api)

100%|██████████| 3/3 [00:05<00:00,  1.73s/it]


In [612]:
#plot_path = get_numerical_path(ibea_50_1_path,vehicles=vehicles)

test2 = plot_routes_with_congestion_folium(fixed_path,points_coordinate,dbf,colors=color_map_2,station_ids = True,congestion_edge=congestion_edge,locked_paths=locked_paths, prerocessed_paths=preprocess_fixed_congestion,here_api=True, api_key=api)
test2

  0%|          | 0/1 [00:00<?, ?it/s]

[[55.7743458 -3.9184134]
 [55.8149158 -3.9376273]]
1 #9467bd
2 #8c564b
1 #e377c2


100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


In [608]:
#plot_path = get_numerical_path(ibea_50_1_path,vehicles=vehicles)

test3 = plot_routes_with_congestion_folium(fixed_path,points_coordinate,dbf,colors=color_map_2,station_ids = True,congestion_edge=congestion_edge, prerocessed_paths=preprocess_fixed_congestion,here_api=True, api_key=api)
test3

[[55.7743458 -3.9184134]
 [55.8149158 -3.9376273]]


TypeError: 'NoneType' object is not subscriptable

In [613]:
import selenium.webdriver
#test.fit_bounds([[50.7,-1.7],[52,.7]])
#test.fit_bounds([[55,-4],[56.4,-2]])


#max_point =  [max(points_coordinate[:,0])-.1, max(points_coordinate[:,1])]
#min_point =  [min(points_coordinate[:,0])+.1, min(points_coordinate[:,1])-0.6]
#test2.position = np.mean([points_coordinate[congestion_edge[0]], points_coordinate[congestion_edge[1]]],axis=0)#([min_point,max_point])
test2.save("map.html")
driver = selenium.webdriver.Chrome()
driver.set_window_size(1500,1500)  # choose a resolution
driver.get('file:///home/andreash/Documents/School/Master_Thesis/Explorer/src/map.html')
time.sleep(1)
driver.save_screenshot('screenshot25_test_congestion.png')

True